In [ ]:
from typing import Tuple
import pandas as pd
import numpy as np

from pathlib import Path
from matplotlib import pyplot, image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
def returnAnnotations(file_attributos, file_annotations):
  df_attributes = pd.read_csv(file_attributos, header = None)
  list_attributes = list()
  list_attributes.append(df_attributes[0].values)
  handle = open(file_annotations)
  scores = dict()
  confidence = dict()
  for line in handle:
      words = line.split()
      i=0
      for word in words:
          if i == 0:
              scores[word]=list()
              confidence[word] = list()
              i+=1
          else:
              values = word.split(',')
              scores[words[0]].append(float(values[0]))
              confidence[words[0]].append(float(values[1]))
              i+=1
  scores = pd.DataFrame(scores).T
  scores.columns = list_attributes
  confidence = pd.DataFrame(confidence).T
  confidence.columns = list_attributes
  return scores,confidence


In [ ]:
file_attributos = '/content/drive/My Drive/courses/mdc/dados/trabalho_final/Predição de Atributos Visuais/annotations/attributes.txt'
file_annotations = '/content/drive/My Drive/courses/mdc/dados/trabalho_final/Predição de Atributos Visuais/annotations/annotations.tsv'
file_training = '/content/drive/My Drive/courses/mdc/dados/trabalho_final/Predição de Atributos Visuais/holdout_split/training.txt'
file_test = '/content/drive/My Drive/courses/mdc/dados/trabalho_final/Predição de Atributos Visuais/holdout_split/test.txt'
def returnTrainingTest(file_attributos, file_annotations, file_training, file_test):
  scores,confidence = returnAnnotations(file_attributos, file_annotations)
  df_training = pd.read_csv(file_training, header = None)
  df_test = pd.read_csv(file_test, header = None)
  scores_training = scores[scores.index.isin(df_training[0].values)]
  scores_test = scores[scores.index.isin(df_test[0].values)]
  confidence_training = confidence[confidence.index.isin(df_training[0].values)]
  confidence_test = confidence[confidence.index.isin(df_test[0].values)]
  return scores_training, scores_test, confidence_training, confidence_test

In [ ]:
scores_training, scores_test, confidence_training, confidence_test = returnTrainingTest(file_attributos, file_annotations, file_training, file_test)

In [ ]:
scores_training

In [ ]:
scores_test

In [ ]:
confidence_training

In [ ]:
confidence_test

In [ ]:
confidence_training = confidence_training.apply(lambda x: [0 if y <= 5 else 1 for y in x])
confidence_training

In [ ]:
confidence_test = confidence_test.apply(lambda x: [0 if y <= 5 else 1 for y in x])
confidence_test

In [ ]:
#df_training = pd.DataFrame(scores_training*confidence_training, columns=scores_training.columns, index=scores_training.index)
df_training = scores_training
df_training.columns = df_training.columns.get_level_values(0)
df_training

In [ ]:
#df_test = pd.DataFrame(scores_test*confidence_test, columns=scores_test.columns, index=scores_test.index)
df_test = scores_test
df_test.columns = df_test.columns.get_level_values(0)
df_test

In [ ]:
path_img = Path('/content/drive/My Drive/courses/mdc/dados/trabalho_final/Predição de Atributos Visuais/imageLD')
import os
def returnPathImages():
    imageLD = {}
    datasets = os.listdir(path_img)
    for dataset in datasets:
        imgs_name = os.listdir(path_img/Path(dataset))
        for name in imgs_name:
            img_file = path_img/Path(dataset)/Path(name)
            imageLD[str(Path(dataset)/Path(name))]=img_file
    return imageLD

In [ ]:
images_paths = returnPathImages()

In [ ]:
df_training_image = df_training[df_training.index.isin(images_paths.keys())]
df_training_image

In [ ]:
df_test_image = df_test[df_test.index.isin(images_paths.keys())]
df_test_image

In [ ]:
images_paths_train = {k: v for k, v in images_paths.items() if k in df_training.index}

In [ ]:
images_paths_teste = {k: v for k, v in images_paths.items() if k in df_test.index}

In [ ]:
data_treino = df_training.copy()
data_treino['path'] = df_training.index

In [ ]:
data_teste = df_test.copy()
data_teste['path'] = df_test.index

Sem data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
datagen = ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input)

In [ ]:
img_height = 224
img_width = 224
BATCH_SIZE = 20

In [ ]:
train_generator = datagen.flow_from_dataframe(dataframe=data_treino, directory=path_img,
                                              target_size=(img_height, img_width),
                                              x_col='path',
                                              y_col=data_treino.columns[:-1],
                                              batch_size=BATCH_SIZE,
                                              class_mode="raw",
                                              subset='training',
                                              shuffle=True,
                                              seed=7)

validation_generator = datagen.flow_from_dataframe(dataframe=data_treino, directory=path_img,
                                             target_size=(img_height, img_width),
                                             x_col='path',
                                             y_col=data_treino.columns[:-1],
                                             batch_size=1,
                                             class_mode="raw",
                                             subset='validation',
                                             shuffle=True,
                                             seed=7)

test_generator = datagen.flow_from_dataframe(dataframe=data_teste, directory=path_img,
                                             target_size=(img_height, img_width),
                                             x_col='path',
                                             y_col=data_treino.columns[:-1],
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=7)

In [ ]:
import tensorflow as tf
from keras import layers
from keras.applications import VGG16
from keras.models import Model
pre_trained_model  = VGG16(include_top=False, input_shape=(img_height,img_width,3), weights='imagenet')
transfer_layer = pre_trained_model.get_layer('block5_pool')
conv_model = Model(inputs=pre_trained_model.input, outputs=transfer_layer.output)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
# Start a new Keras Sequential model.
model = Sequential()

# Add the convolutional part of the VGG16 model from above.
model.add(conv_model)
model.add(BatchNormalization())
# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
model.add(Dropout(0.1))

# Add the final layer for the actual classification.
labels = 40
model.add(Dense(labels, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes = True)

In [ ]:
input_shape = pre_trained_model.layers[0].output_shape[0][1:3]
input_shape

In [ ]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
  SS_res =  K.sum(K.square( y_true-y_pred ))
  SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
  return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
model.compile(optimizer = optimizer, loss='mean_absolute_error', metrics=[coeff_determination])

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

In [ ]:
from keras import utils, callbacks
from keras.callbacks import LearningRateScheduler
earlystopping = callbacks.EarlyStopping(monitor='val_loss', mode='min', patience= 15, restore_best_weights=True)
epochs = 250

In [ ]:
total_train = len(train_generator.filenames)
total_val = len(validation_generator.filenames)
history = model.fit(train_generator, epochs=epochs, steps_per_epoch=total_train//BATCH_SIZE, validation_data=validation_generator, validation_steps=total_val, verbose=1, callbacks=[LearningRateScheduler(lr_schedule),earlystopping])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b--', label = 'MAE de Treino')
plt.plot(history.history['val_loss'], 'r-.', label = 'MAE de Validação')
plt.xlabel('Epochs')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['coeff_determination'], 'b--', label = 'R2 de Treino')
plt.plot(history.history['val_coeff_determination'], 'r-.', label = 'R2 de Validação')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
model.save_weights('weights.h5')
